In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *
import seaborn as sns

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
## Load everything
filename = 'Source_cats/LAE_50000_z1.9-7/'

pm_flx = np.load(filename + 'pm_flx.npy')
pm_flx_noerr = np.load(filename + 'pm_flx_no_line_no_err.npy')

mock = {}

aux = []
with open(filename + 'SEDs.csv') as csv_file:
    rdr = csv.reader(csv_file, delimiter=',')
    for row in rdr:
        aux.append(row)
mock['SEDs'] = np.array(aux, dtype=float)
aux = []
with open(filename + 'SEDs_no_line.csv') as csv_file:
    rdr = csv.reader(csv_file, delimiter=',')
    for row in rdr:
        aux.append(row)
mock['SEDs_no_line'] = np.array(aux, dtype=float)

utils = np.load(filename + 'utils.npy', allow_pickle=True).item()

mock['redshift_Lya_Arr'] = utils['z_Arr']
mock['w_Arr'] = utils['w_Arr']
mock['EW_Arr'] = utils['EW_Arr']
mock['L_Arr'] = utils['L_Arr']

## Somehow, some bright mocks are sneaking in. So, get rid of them.
mask_bright = np.where(pm_flx_noerr[-3] <= 1e-16)[0]
pm_flx_noerr = pm_flx_noerr[:, mask_bright]
pm_flx = pm_flx[:, mask_bright]
for key in mock.keys():
    if key == 'w_Arr': continue
    mock[key] = mock[key][mask_bright]

err_fit_params = np.load('npy/err_fit_params_minijpas.npy')
m = err_fit_params[:, 0].reshape(-1, 1)
b = err_fit_params[:, 1].reshape(-1, 1)
pm_err = np.load(filename + 'pm_flx_err.npy')[:, mask_bright]

w_central = central_wavelength()

In [ ]:
EW_lya = mock['EW_Arr']

zspec = mock['redshift_Lya_Arr']

In [ ]:
w_lya = 1215.67 # A
N_sources = pm_flx.shape[1]
N_sources

In [ ]:
mag = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
mag[np.isnan(mag)] = 99.

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)

In [ ]:
## Cuts
mag_cut_min = 20
mag_cut_max = 23

z_cut_min = 2.5
z_cut_max = 4

mag_cut = (mag[-2] > mag_cut_min) & (mag[-2] < mag_cut_max)

target_z = (zspec > z_cut_min) & (zspec < z_cut_max)
target = (
    mag_cut & target_z & (EW_lya / (1 + zspec) > 20)
)

## Extract lines
ew0min_lya = 30
ew0min_oth = 150

line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other, ew0min_oth,
    obs=True)
other_lines = identify_lines(line_other, pm_flx, pm_err, first=False)

line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0min_lya)
lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)

## Compute z
z_nb_Arr = np.ones(N_sources) * -1 
z_nb_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

## Select
line_len = np.array([len(l) for l in other_lines])
nice_lya = (np.array(lya_lines) != -1) & (line_len <= 1)
nice_z = np.abs(z_nb_Arr - zspec) < 0.12

z_nb_cut = (z_nb_Arr > z_cut_min) & (z_nb_Arr < z_cut_max)

selected = mag_cut & nice_lya & z_nb_cut
select = len(np.where(selected)[0])
rightz = len(np.where(selected & nice_z)[0])

N_target2 = len(np.where(target)[0])

ew0_Arr = pm_flx[lya_lines] * 150 / cont_est_lya\
    / (1 + z_nb_Arr)

selectg = np.histogram2d(ew0_Arr[selected], cont_est_lya[selected])
rightzg = np.histogram2d(ew0_Arr[selected & nice_z], cont_est_lya[selected & nice_z])

purityg = rightzg / selectg
completenessg = rightzg / N_target2

In [ ]:
fig = plt.figure(figsize=(8, 8))

width = 0.5
height = 0.5
spacing = 0.06
cbar_width = 0.05
print('ok1')

# Define axes
ax00 = fig.add_axes([0, height + 1.5 * spacing, width, height])
ax01 = fig.add_axes([width + spacing, height + 1.5 * spacing, width, height], sharey=ax00)
ax10 = fig.add_axes([0, 0, width, height], sharex=ax00)
ax11 = fig.add_axes([width + spacing, 0, width, height], sharex=ax01, sharey=ax10)
axcbar0 = fig.add_axes([2 * width + 1.5 * spacing, height + 1.5 * spacing, cbar_width, height])
axcbar1 = fig.add_axes([2 * width + 1.5 * spacing, 0, cbar_width, height])

# Plot stuff in the rectangles
vmax = np.max([np.max(rightzg), np.max(selectg)])
cmap = 'Spectral'

sns.heatmap(rightzg, ax=ax00, vmin=0, vmax=vmax, cbar_ax=axcbar0, cmap=cmap)
sns.heatmap(selectg, ax=ax01, vmin=0, vmax=vmax, cbar_ax=axcbar0, cmap=cmap)

sns.heatmap(purityg, ax=ax10, vmin=0, vmax=1, cbar_ax=axcbar1)
sns.heatmap(completenessg, ax=ax11, vmin=0, vmax=1, cbar=False)

ax00.invert_yaxis()
ax10.invert_yaxis()
print('ok2')

# Axes ticks
#xticks = range(ew0oth_step)
#yticks = range(ew0lya_step)
#xtick_labels = ['{0:0.0f}'.format(n) for n in np.linspace(ew0oth_min, ew0oth_max, ew0oth_step)]
#ytick_labels = ['{0:0.0f}'.format(n) for n in np.linspace(ew0lya_min, ew0lya_max, ew0lya_step)]

#ax00.set_yticks(yticks)
#ax00.set_yticklabels(ytick_labels)
#ax00.set_xticks(xticks)
#ax00.set_xticklabels(xtick_labels, rotation='vertical')

#ax10.set_yticks(yticks)
#ax10.set_yticklabels(ytick_labels)
#ax10.set_xticks(xticks)
#ax10.set_xticklabels(xtick_labels, rotation='vertical')

#ax11.set_xticks(xticks)
#ax11.set_xticklabels(xtick_labels, rotation='vertical')
#ax11.set_yticks(yticks)
#ax11.set_yticklabels(ytick_labels)

#ax01.set_xticks(xticks)
#ax01.set_xticklabels(xtick_labels, rotation='vertical')
#ax01.set_yticks(yticks)
#ax01.set_yticklabels(ytick_labels)

# Axes labels
ylabel = r'Ly$\alpha$ EW$_0$ ($\AA$)'
xlabel = 'Other lines EW$_\mathrm{obs}$ ($\AA$)'
ax00.set_ylabel(ylabel, fontsize=12)
ax10.set_ylabel(ylabel, fontsize=12)
ax10.set_xlabel(xlabel, fontsize=12)
ax11.set_xlabel(xlabel, fontsize=12)

# Set titles
ax00.set_title('Selected w/ correct z', fontsize=15)
ax01.set_title('All selected', fontsize=15)
ax10.set_title('Purity', fontsize=15)
ax11.set_title('Completeness', fontsize=15)

plt.savefig('output/puri-comp_magcut-' + str(mag_cut) + '.pdf', dpi=600,
    bbox_inches='tight')
plt.show()

print('N_target = {}'.format(N_target))